In [2]:
import os

from pathling import PathlingContext
from pyspark.sql.functions import lit

MIMIC_FHIR_PATH="../_data/mimic4-demo-ptl"

In [3]:
os.environ['SPARK_CONF_DIR'] = os.path.abspath('../env/spark-conf')

In [4]:
pc = PathlingContext.create()

:: loading settings :: url = jar:file:/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/szu004/.ivy2/cache
The jars for the packages stored in: /Users/szu004/.ivy2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-53020d33-f090-4f99-9626-52562211d9be;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT in local-m2-cache
	found io.delta#delta-spark_2.12;3.2.0 in local-m2-cache
	found io.delta#delta-storage;3.2.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
	found org.apache.hadoop#hadoop-aws;3.3.4 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in local-m2-cache
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
:: resolution report :: resolve 121ms :: artifacts dl 7ms
	:: modules in use:
	au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT from local-m2-cache in [default]
	co

2024-09-02 16:57:01 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-09-02 16:57:02 INFO  Version:21 - HV000001: Hibernate Validator null
2024-09-02 16:57:02 DEBUG AbstractConfigurationImpl:176 - Setting custom MessageInterpolator of type au.csiro.pathling.shaded.org.hibernate.validator.messageinterpolation.ParameterMessageInterpolator
2024-09-02 16:57:02 DEBUG ValidationXmlParser:122 - Trying to load META-INF/validation.xml for XML based Validator configuration.
2024-09-02 16:57:02 DEBUG ResourceLoaderHelper:61 - Trying to load META-INF/validation.xml via TCCL
2024-09-02 16:57:02 DEBUG ResourceLoaderHelper:67 - Trying to load META-INF/validation.xml via Hibernate Validator's class loader
2024-09-02 16:57:02 DEBUG ValidationXmlParser:129 - No META-INF/validation.xml found. Using annotation based configuration only.
2024-09-02 16:57:02 DEBUG TraversableResolvers:61 - Cannot find jakarta.persistence.Per

In [5]:
spark = pc.spark

In [6]:
print('spark.sql.ansi.enabled', spark.conf.get("spark.sql.ansi.enabled"))
print('spark.sql.timestampType', spark.conf.get("spark.sql.timestampType"))

spark.sql.ansi.enabled false
spark.sql.timestampType TIMESTAMP_NTZ


In [7]:
data = pc.read.parquet(MIMIC_FHIR_PATH)
data.read('Patient').show()

2024-09-02 16:57:05 WARN  SparkStringUtils:72 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+----+----------+--------------------+
|                  id|        id_versioned|                meta|implicitRules|language|text|          identifier|active|                name|telecom|gender| birthDate|deceasedBoolean|deceasedDateTime|address|       maritalStatus|multipleBirthBoolean|multipleBirthInteger|photo|contact|       communication|generalPractitioner|managingOrganization|link|      _fid|          _extension|
+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+-

## icustay_detail

### Patient

In [8]:
rv_patient = data.view('Patient', 
    select = [
        {
            "column": [
                {
                    "name": "patient_id",
                    "path": "getResourceKey()",
                    "collection": False
                },
                {
                    "name": "gender",
                    "path": "gender",
                    "collection": False
                },
                {
                    "name": "race_system",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).system",
                    "collection": False
                },
                {
                    "name": "race_code",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).code",
                    "collection": False
                },
                {
                    "name": "ethnicity_system",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension('ombCategory').value.ofType(Coding).system",
                    "collection": False
                },
                {
                    "name": "ethnicity_code",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('ombCategory').value.ofType(Coding).code",
                    "collection": False
                },
               
            ]
        }
    ]
)

rv_patient.createOrReplaceTempView('rv_patient')
rv_patient.toPandas()

,patient_id,gender,race_system,race_code,ethnicity_system,ethnicity_code
0,Patient/28dcf33b-0c52-587f-83ad-2a3270976719,female,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
1,Patient/74a2fd87-885b-5eca-9f8b-9141915dba51,female,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
2,Patient/51d2190c-cc46-56c5-b2ea-363895cbea75,male,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
3,Patient/c4c29979-f2f5-56db-af5b-1715887727b8,male,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
4,Patient/b410dd44-7d65-56f9-974f-2751e8aa80e2,male,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
...,...,...,...,...,...,...
95,Patient/dc332fb6-61f8-552e-8743-765d626f0a4e,male,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
96,Patient/df756e08-6ea8-5d69-b918-67911945f827,female,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
97,Patient/af0e5009-d87d-52a8-ac8a-676e471c41f1,male,urn:oid:2.16.840.1.113883.6.238,2106-3,urn:oid:2.16.840.1.113883.6.238,2106-3
98,Patient/568cb149-804c-59e8-bdf5-816e8151cd22,male,urn:oid:2.16.840.1.113883.6.238,2054-5,urn:oid:2.16.840.1.113883.6.238,2054-5


In [9]:
dv_patient = spark.sql("""
SELECT patient_id AS subject_id,
    CASE 
        WHEN gender = 'male' THEN 'M'
        WHEN gender = 'female' THEN 'F'
    END AS gender,
    CASE 
        WHEN race_system = 'urn:oid:2.16.840.1.113883.6.238' THEN
            CASE 
                WHEN race_code = '2106-3' THEN 
                    CASE
                        WHEN ethnicity_system = 'urn:oid:2.16.840.1.113883.6.238' AND ethnicity_code = '2135-2' THEN 'HISPANIC' 
                        ELSE 'WHITE'
                    END
                WHEN race_code = '2054-5' THEN 'BLACK'
                WHEN race_code = '2028-9' THEN 'ASIAN'
            END
    END AS race
FROM rv_patient    
""")
dv_patient.createOrReplaceTempView('dv_patient')
dv_patient.toPandas()

,subject_id,gender,race
0,Patient/28dcf33b-0c52-587f-83ad-2a3270976719,F,WHITE
1,Patient/74a2fd87-885b-5eca-9f8b-9141915dba51,F,WHITE
2,Patient/51d2190c-cc46-56c5-b2ea-363895cbea75,M,WHITE
3,Patient/c4c29979-f2f5-56db-af5b-1715887727b8,M,WHITE
4,Patient/b410dd44-7d65-56f9-974f-2751e8aa80e2,M,WHITE
...,...,...,...
95,Patient/dc332fb6-61f8-552e-8743-765d626f0a4e,M,WHITE
96,Patient/df756e08-6ea8-5d69-b918-67911945f827,F,WHITE
97,Patient/af0e5009-d87d-52a8-ac8a-676e471c41f1,M,WHITE
98,Patient/568cb149-804c-59e8-bdf5-816e8151cd22,M,BLACK


### Encounter(ICU) 


In [10]:
rv_icu_encounter = data.view('Encounter',
                    select = [
                        {
                            "column": [
                                {
                                    "name": "encounter_id",
                                    "path": "getResourceKey()",
                                    "type": "string",
                                    "collection": False
                                },
                                {
                                    "name": "patient_id",
                                    "path": "subject.getReferenceKey()",
                                    "type": "string",
                                    "collection": False
                                },
                                {
                                    "name": "admit_date",
                                    "path": "period.start",
                                    "type": "dateTime",
                                    "collection": False
                                },
                                {
                                    "name": "disch_date",
                                    "path": "period.end",
                                    "type": "dateTime",
                                    "collection": False
                                }

                            ]
                        }
                    ],
                    where= [
                        {
                            "path":"class.system = 'http://terminology.hl7.org/CodeSystem/v3-ActCode'"
                        },
                        {
                            "path":"class.code = 'ACUTE'"
                        },

                    ]
                    )

rv_icu_encounter.createOrReplaceTempView('rv_icu_encounter')
rv_icu_encounter.toPandas()

,encounter_id,patient_id,admit_date,disch_date
0,Encounter/aa9dc967-aece-541b-89d6-b0cedcd43d64,Patient/71a27b6b-5e41-509f-83a8-8f47e8fb8d78,2154-04-24T23:03:44+00:00,2154-05-02T15:55:21+00:00
1,Encounter/9a932a63-68cd-53b0-96f4-6a21cd087156,Patient/aeda7084-be35-510e-b171-f00596aed99f,2169-01-15T04:56:00+00:00,2169-01-20T15:47:50+00:00
2,Encounter/55e72ffc-3f32-5738-891d-15076c9fc1b7,Patient/aeda7084-be35-510e-b171-f00596aed99f,2170-02-24T18:18:46+00:00,2170-02-25T15:15:26+00:00
3,Encounter/b34e83f0-863c-533f-a5c0-84fcb98c20bb,Patient/f77a5b72-65fd-5b20-8cef-6b6be4791265,2176-11-26T02:34:49+00:00,2176-11-29T20:58:54+00:00
4,Encounter/10a15b4a-8adf-5087-9e41-cc2c3d856026,Patient/8adbf3e4-47ff-561e-b1b6-746ee32e056d,2150-06-03T20:12:32+00:00,2150-06-04T21:05:58+00:00
...,...,...,...,...
135,Encounter/0667da78-0d04-571d-aa1a-02f72998b144,Patient/cd462e42-c070-5235-ae76-c37733a451be,2189-06-09T12:46:30+00:00,2189-06-10T22:58:09+00:00
136,Encounter/8076633f-832c-5619-aa91-8669a7e68e1d,Patient/8adbf3e4-47ff-561e-b1b6-746ee32e056d,2150-03-11T15:34:56+00:00,2150-03-19T02:17:47+00:00
137,Encounter/c0229b65-f599-5108-b1ba-94bd1ca7b94f,Patient/1bb918ba-e04e-5e7a-87ca-dbcbbb4c72c3,2143-03-22T06:42:00+00:00,2143-03-25T15:05:33+00:00
138,Encounter/e7c2a847-825a-5434-b7f7-c409ae22b8e4,Patient/fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,2178-12-21T06:05:18+00:00,2178-12-22T02:16:08+00:00


In [11]:
dv_icu_encounter= spark.sql("""
WITH tv AS (
    SELECT 
        patient_id AS subject_id,
        encounter_id AS stay_id,
        CAST(admit_date AS TIMESTAMP) icu_intime, 
        CAST(disch_date  AS TIMESTAMP) icu_outtime 
    FROM rv_icu_encounter)
SELECT *, 
    row_number() over (PARTITION BY subject_id, stay_id ORDER BY icu_intime ASC) AS icustay_seq 
FROM tv
""")
dv_icu_encounter.createOrReplaceTempView('dv_icu_encounter')
dv_icu_encounter.toPandas()

,subject_id,stay_id,icu_intime,icu_outtime,icustay_seq
0,Patient/0a8eebfd-a352-522e-89f0-1d4a13abdebc,Encounter/18f74cab-e07d-5ca6-98b8-01f8f9956caa,2180-07-23 14:00:00,2180-07-23 23:50:47,1
1,Patient/0c2243d2-987b-5cbd-8eb1-170a80647693,Encounter/31c84afa-feb0-5634-959d-a0cc62fe06bd,2149-10-02 12:48:08,2149-10-04 17:48:36,1
2,Patient/13df78e7-150e-5eb7-be5f-5f62b2baee87,Encounter/a50796c8-9e4a-5272-857b-ea06967e6b7b,2177-03-15 05:49:26,2177-03-15 22:47:09,1
3,Patient/158f3a39-e3d7-5e7a-93aa-57af894aadd9,Encounter/497d61a0-1f48-5229-ac51-5ab4bded1cb3,2144-10-29 23:09:03,2144-11-02 15:24:29,1
4,Patient/1ab119a5-aac8-5002-9d2f-b8ff69623387,Encounter/13a20e28-a0e6-5a15-825e-1e78c7eaaacb,2148-09-10 13:19:00,2148-09-15 21:50:29,1
...,...,...,...,...,...
135,Patient/f5a99b42-365c-5f7e-bcab-4fa33b3cc88e,Encounter/fe9b006e-e9db-562e-a2ae-860e4e636574,2176-12-22 16:24:33,2176-12-23 20:14:53,1
136,Patient/f5efdf3f-5b53-5c9f-95a6-047275107c46,Encounter/25e05468-7cbf-5a04-9209-79cb07703326,2141-05-22 20:18:01,2141-05-27 22:24:02,1
137,Patient/f77a5b72-65fd-5b20-8cef-6b6be4791265,Encounter/b34e83f0-863c-533f-a5c0-84fcb98c20bb,2176-11-26 02:34:49,2176-11-29 20:58:54,1
138,Patient/fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,Encounter/042aabd7-5383-5531-ac04-574078813c7e,2177-07-14 20:38:00,2177-07-15 16:08:36,1


### ICU Stay Detail

In [12]:
icustay_detail = spark.sql("""
SELECT 
    enc.subject_id, enc.stay_id, pat.gender, pat. race, enc.icu_intime, enc.icu_outtime, enc.icustay_seq 
FROM dv_icu_encounter enc
LEFT OUTER JOIN dv_patient pat ON enc.subject_id = pat.subject_id
""")
icustay_detail.toPandas()
icustay_detail.write.parquet('../_data/mimic_derrived/icustay_detail.parquet', mode='overwrite')

## Observations 
### Chartevents

In [13]:
def obs_for_coding(system, code, view_name = None):
    obs_view = data.view('Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "date",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueQuantity.value",
                                         "type": "decimal",
                                         "collection": False
                                     },
                                     {
                                         "name": "unit",
                                         "path": "valueQuantity.unit",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":f"code.coding.system = '{system}'"
                             },
                             {
                                 "path":f"code.coding.code = '{code}'"
                             },

                         ]
                         )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view


CS_MIMIC_CHARTEVENTS_D_ITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'
CS_MIMIC_D_LABITEMS = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'

def obs_chartevent_for_code(code, view_name = None):
    return obs_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, code, view_name)

def obs_labitems_for_code(code, view_name = None):
    return obs_for_coding(CS_MIMIC_D_LABITEMS, code, view_name)

In [14]:
obs_resp_rate = obs_chartevent_for_code('224690', 'obs_resp_rate')
obs_resp_rate.show()

+--------------------+--------------------+--------------------+--------------------+---------+--------+
|                  id|          patient_id|        encounter_id|                date|    value|    unit|
+--------------------+--------------------+--------------------+--------------------+---------+--------+
|fcae13c7-ddd9-5d4...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T00:00:...|18.000000|insp/min|
|a3c20e0d-9c7b-5a6...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T07:00:...|14.000000|insp/min|
|b22b0632-3352-505...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T11:00:...|14.000000|insp/min|
|5c457b43-a9fc-5f7...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T15:00:...|14.000000|insp/min|
|f2b42818-fa58-5fb...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T20:00:...|16.000000|insp/min|
|1ba6f7af-a172-54c...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-25T00:00:...|14.000000|insp/min|
|bdd48277-3cea-53b...|Patient/2955c958-...|Encounter/84

In [15]:
obs_spo2 = obs_chartevent_for_code('220277', 'obs_spo2')
obs_spo2.show()

+--------------------+--------------------+--------------------+--------------------+----------+----+
|                  id|          patient_id|        encounter_id|                date|     value|unit|
+--------------------+--------------------+--------------------+--------------------+----------+----+
|3cfa7064-aba9-596...|Patient/2955c958-...|Encounter/2b6a330...|2178-07-05T07:42:...|100.000000|   %|
|0e2e6c75-581a-5fc...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T00:00:...|100.000000|   %|
|0b5b5e1f-16e6-5e4...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T01:00:...| 96.000000|   %|
|95b2b799-be1f-533...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T02:00:...| 92.000000|   %|
|048f13c1-fd5a-555...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T03:00:...|100.000000|   %|
|4dc19db8-4f29-577...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T04:00:...|100.000000|   %|
|ccf50729-4b79-511...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T05:00:.

In [16]:
obs_heart_rate = obs_chartevent_for_code('220045', 'obs_heart_rate')
obs_heart_rate.show()

+--------------------+--------------------+--------------------+--------------------+----------+----+
|                  id|          patient_id|        encounter_id|                date|     value|unit|
+--------------------+--------------------+--------------------+--------------------+----------+----+
|d35bbfbf-73bf-5b5...|Patient/2955c958-...|Encounter/2b6a330...|2178-07-05T07:42:...|103.000000| bpm|
|072fd1e4-6b01-517...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T00:00:...| 84.000000| bpm|
|a30dce5c-48ae-5a3...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T01:00:...| 87.000000| bpm|
|121ee7e1-2ef1-5cd...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T02:00:...| 89.000000| bpm|
|a1313740-5fc4-533...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T03:00:...| 81.000000| bpm|
|211e2401-d817-5d1...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T04:00:...| 87.000000| bpm|
|d36d5b72-7dcb-536...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T05:00:.

In [17]:
#
# alike mimiciv_derrived.vital_signs
#
derr_vital_signs_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS resp_rate, NULL AS heart_rate, NULL AS spo2 FROM obs_resp_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, value AS heart_rate,  NULL AS spo2 FROM obs_heart_rate 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS resp_rate, NULL AS heart_rate,  value AS spo2 FROM obs_spo2 
) 
SELECT patient_id, encounter_id, obs_time, FIRST(resp_rate) AS resp_rate, FIRST(heart_rate) AS heart_rate, FIRST(spo2) AS spo2 FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_vital_signs_view.sample(0.02).toPandas()

,patient_id,encounter_id,obs_time,resp_rate,heart_rate,spo2
0,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24 00:00:00,18.000000,None,None
1,Patient/4365e125-c049-525a-9459-16d5e6947ad2,Encounter/b742bb23-5e32-5ae2-ba52-37e21e070765,2116-12-11 16:00:00,18.000000,None,None
2,Patient/a2605b15-4f1b-5839-b4ce-fb7a6bc1005f,Encounter/5041d382-c2f6-570f-a28b-9511135d47bf,2135-01-19 13:00:00,17.000000,None,None
3,Patient/73fb53d8-f1fa-53cd-a25c-2314caccbb99,Encounter/45205f70-733f-58c2-a43e-790d64a7095d,2146-07-08 23:00:00,21.000000,None,None
4,Patient/73fb53d8-f1fa-53cd-a25c-2314caccbb99,Encounter/45205f70-733f-58c2-a43e-790d64a7095d,2146-07-03 07:00:00,21.000000,None,None
...,...,...,...,...,...,...
303,Patient/77e10fd0-6a1c-5547-a130-fae1341acf36,Encounter/af3040c6-eb23-59c8-9ad0-818a93f08d86,2137-08-30 11:02:00,None,None,97.000000
304,Patient/77e10fd0-6a1c-5547-a130-fae1341acf36,Encounter/af3040c6-eb23-59c8-9ad0-818a93f08d86,2137-08-19 17:19:00,None,None,99.000000
305,Patient/8adbf3e4-47ff-561e-b1b6-746ee32e056d,Encounter/8076633f-832c-5619-aa91-8669a7e68e1d,2150-03-11 15:51:00,None,None,96.000000
306,Patient/8e77dd0b-932d-5790-9ba6-5c6df8434457,Encounter/84fd948e-379e-5c37-86bf-3c9bca4835b5,2140-01-24 18:02:00,None,None,97.000000


### Labevents

In [18]:
# bg so2
obs_bg_so2 = obs_labitems_for_code('50817', 'obs_bg_so2')
# AND < 100
obs_bg_so2.show()

+--------------------+--------------------+--------------------+--------------------+---------+----+
|                  id|          patient_id|        encounter_id|                date|    value|unit|
+--------------------+--------------------+--------------------+--------------------+---------+----+
|69ca4208-7476-593...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T22:17:...|98.000000|   %|
|34706e68-6ab5-505...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T05:23:...|97.000000|   %|
|64ff5dcd-f26c-556...|Patient/bc2a74ce-...|                NULL|2140-10-03T07:15:...|74.000000|   %|
|2ff0afd6-942c-579...|Patient/bc2a74ce-...|Encounter/b7566a4...|2140-10-03T09:25:...|73.000000|   %|
|0d18961c-e1db-59f...|Patient/bc2a74ce-...|                NULL|2137-06-19T15:48:...|73.000000|   %|
|550dd506-7fd2-510...|Patient/c3412a5f-...|Encounter/5ee5f4a...|2119-10-26T07:22:...|97.000000|   %|
|26725c70-7885-5c2...|Patient/c3412a5f-...|                NULL|2119-10-26T06:25:...|48.000

In [19]:
# bg so2
obs_bg_pco2 = obs_labitems_for_code('50818', 'obs_bg_pco2')
obs_bg_pco2.show()

+--------------------+--------------------+--------------------+--------------------+---------+-----+
|                  id|          patient_id|        encounter_id|                date|    value| unit|
+--------------------+--------------------+--------------------+--------------------+---------+-----+
|bbe0d98c-f367-570...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T00:10:...|38.000000|mm Hg|
|11f6e65e-e072-5dd...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T22:17:...|39.000000|mm Hg|
|023811f9-620c-545...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T05:23:...|35.000000|mm Hg|
|1bbf8896-ec5d-5f7...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T16:34:...|39.000000|mm Hg|
|19333fc0-08ad-54f...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T13:46:...|47.000000|mm Hg|
|fa450e6f-0ba2-529...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T19:49:...|50.000000|mm Hg|
|cf6412d3-def6-50a...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T02:53:.

In [20]:
# hemoglobin
obs_bg_hemoglobin = obs_labitems_for_code('50811', 'obs_bg_hemoglobin')
obs_bg_hemoglobin.toPandas()

2024-09-02 16:57:13 WARN  GarbageCollectionMetrics:72 - To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


,id,patient_id,encounter_id,date,value,unit
0,1236173a-d92a-500e-b3a5-62ce620237c0,Patient/bbad4581-d089-54a7-b7a0-8d986c5fb5ec,Encounter/508ab45c-85c2-59f8-856b-5fba2186804c,2167-05-05T16:34:00+00:00,10.900000,g/dL
1,a88475dd-25e1-50b1-8784-6cd125486704,Patient/bbad4581-d089-54a7-b7a0-8d986c5fb5ec,Encounter/508ab45c-85c2-59f8-856b-5fba2186804c,2167-05-05T13:46:00+00:00,14.100000,g/dL
2,4f4e7911-5303-50d3-90ab-d6952623145c,Patient/bbad4581-d089-54a7-b7a0-8d986c5fb5ec,Encounter/508ab45c-85c2-59f8-856b-5fba2186804c,2167-05-05T17:57:00+00:00,11.000000,g/dL
3,1f6fd236-270b-5287-a2d7-b14baf22f552,Patient/c0d83ad6-0b65-5fd6-b241-f50cf552232c,Encounter/4125afae-262f-5d34-a157-9f4ef0ef09de,2180-02-21T08:02:00+00:00,16.000000,g/dL
4,5546a723-83ce-590f-bb7f-1f24bdeefe3f,Patient/c0d83ad6-0b65-5fd6-b241-f50cf552232c,Encounter/4125afae-262f-5d34-a157-9f4ef0ef09de,2180-02-21T10:15:00+00:00,12.500000,g/dL
...,...,...,...,...,...,...
138,b8cdc1f6-2658-521b-89ad-5b5649348bdc,Patient/f5a99b42-365c-5f7e-bcab-4fa33b3cc88e,Encounter/9f012fdd-ce8e-5d6d-a9cf-e8dda4c93b65,2177-03-26T11:47:00+00:00,8.600000,g/dL
139,ce112522-47d8-5f5e-8b57-d4bd6918f697,Patient/f5a99b42-365c-5f7e-bcab-4fa33b3cc88e,Encounter/9a8d3981-0eaa-5f92-8c47-40ffa75340d8,2176-11-19T14:57:00+00:00,12.700000,g/dL
140,2388c8f1-72d8-58e6-8f37-9da456ee54b3,Patient/f5efdf3f-5b53-5c9f-95a6-047275107c46,Encounter/3e802913-a3f3-573f-90b3-a85dffdec47b,2141-05-24T15:07:00+00:00,11.500000,g/dL
141,7eb6bd18-7773-5f7e-b85e-bd69fa2e546a,Patient/f77a5b72-65fd-5b20-8cef-6b6be4791265,None,2175-04-05T06:45:00+00:00,12.000000,g/dL


In [21]:
#
# alike mimiciv_derrived.bg
#
derr_bg_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS so2, NULL AS pco2, NULL AS hemoglobin  FROM obs_bg_so2 
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, value AS pco2, NULL AS hemoglobin FROM obs_bg_pco2
UNION  
SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, NULL AS so2, NULL AS pco2, value AS hemoglobin FROM obs_bg_hemoglobin  
) 
SELECT patient_id, encounter_id, obs_time, FIRST(so2) AS so2, FIRST(pco2) AS pco2, FIRST(hemoglobin) AS hemoglobin FROM ou 
GROUP BY patient_id, encounter_id, obs_time
""")
derr_bg_view.sample(0.02).toPandas()

,patient_id,encounter_id,obs_time,so2,pco2,hemoglobin
0,Patient/27980ad8-77b7-5360-b211-bf4fac9c468c,Encounter/e2ccaeda-5de2-557d-b9b4-b87a9617046d,2135-01-23 01:48:00,37.000000,None,None
1,Patient/a7bbf9a2-f7fe-5815-a637-fa59bd70b374,Encounter/c15395c4-96ce-5511-914b-f443d1aebe68,2111-11-14 02:45:00,None,42.000000,None
2,Patient/c0d83ad6-0b65-5fd6-b241-f50cf552232c,Encounter/4125afae-262f-5d34-a157-9f4ef0ef09de,2180-02-21 19:09:00,None,41.000000,None
3,Patient/a3a12d01-dc21-565b-89e2-da60e7fc80dc,Encounter/2b24741b-d4f8-5a5b-9d60-04f38dd87286,2154-01-03 02:26:00,None,55.000000,None
4,Patient/91e0e410-0782-5478-90e0-1bedc3aa1525,Encounter/4c6a16c0-9bc3-5f7a-8783-0d7693efbcfc,2115-10-04 06:20:00,None,39.000000,None
5,Patient/24450f28-a039-57d8-95c9-d7ba5508ecd4,Encounter/0071a339-74cd-596a-9083-771d41d6d118,2148-01-02 05:26:00,None,38.000000,None
6,Patient/27980ad8-77b7-5360-b211-bf4fac9c468c,Encounter/e2ccaeda-5de2-557d-b9b4-b87a9617046d,2135-01-28 06:59:00,None,55.000000,None
7,Patient/dd2bf984-33c3-5874-8f68-84113327877e,Encounter/05c742f8-5896-5e36-9933-b5c5eca443a1,2133-12-21 02:23:00,None,37.000000,None
8,Patient/4365e125-c049-525a-9459-16d5e6947ad2,Encounter/ddcb961c-5c8a-5548-907e-859de18790db,2116-12-05 08:23:00,None,39.000000,None
9,Patient/24450f28-a039-57d8-95c9-d7ba5508ecd4,Encounter/0071a339-74cd-596a-9083-771d41d6d118,2148-01-04 13:04:00,None,37.000000,None


## Encounters (ICU) 

### Oxygen Delivery 

In [22]:
def obs_str_for_coding(system, code, view_name = None):
    obs_view = data.view('Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "charttime",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "storetime",
                                         "path": "issued",
                                         "type": "instant",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueString",
                                         "type": "string",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":f"code.coding.system = '{system}'"
                             },
                             {
                                 "path":f"code.coding.code = '{code}'"
                             },

                         ]
                         )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view

rv_oxygen_delivery_device = obs_str_for_coding(CS_MIMIC_CHARTEVENTS_D_ITEMS, '226732', 'rv_oxygen_delivery_device')
rv_oxygen_delivery_device.toPandas()

,id,patient_id,encounter_id,charttime,storetime,value
0,51e2b00a-662e-5b9d-9375-8309fe97d23d,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/2b6a3303-8c0e-5483-889e-e4f76461ac17,2178-07-04T15:37:00+00:00,2178-07-05 01:34:00,Tracheostomy tube
1,fe66329e-a0ae-5d01-9bff-764238d0923c,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/2b6a3303-8c0e-5483-889e-e4f76461ac17,2178-07-05T07:37:00+00:00,2178-07-05 17:38:00,Tracheostomy tube
2,62486df1-bc5a-5189-8d83-2d2532450840,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/2b6a3303-8c0e-5483-889e-e4f76461ac17,2178-07-05T11:56:00+00:00,2178-07-05 21:54:00,Tracheostomy tube
3,b0ff0240-2b7f-5891-a65c-af7d0a261413,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24T00:00:00+00:00,2178-07-24 12:29:00,Tracheostomy tube
4,46667ff3-5240-50f8-89e9-c7af0ac06d62,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24T07:00:00+00:00,2178-07-24 17:26:00,Tracheostomy tube
...,...,...,...,...,...,...
3140,3c71d282-76bd-57ad-8788-71f39f1b0ff1,Patient/fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,Encounter/e7c2a847-825a-5434-b7f7-c409ae22b8e4,2178-12-21T12:58:00+00:00,2178-12-21 22:59:00,None
3141,489ae42c-5871-5f13-b0dc-bb6d67d03f18,Patient/fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,Encounter/042aabd7-5383-5531-ac04-574078813c7e,2177-07-15T08:00:00+00:00,2177-07-15 19:38:00,None
3142,4edface3-5b13-5773-b7c3-5fab28022770,Patient/fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,Encounter/042aabd7-5383-5531-ac04-574078813c7e,2177-07-15T12:00:00+00:00,2177-07-15 22:03:00,None
3143,5ca292da-581d-50c9-b37d-1fbea14dcd70,Patient/fa5fbf9c-23e3-5ef3-9cfb-24d20a950314,Encounter/042aabd7-5383-5531-ac04-574078813c7e,2177-07-15T03:20:00+00:00,2177-07-15 14:17:00,None


In [23]:
dv_oxygen_delivery_device = spark.sql("""
WITH tv AS (
    SELECT patient_id, encounter_id, CAST(date AS TIMESTAMP) as obs_time, value AS oxygen_delivery_device FROM rv_oxygen_delivery_device
) 
SELECT *, row_number() OVER (PARTITION BY patient_id, encounter_id, obs_time ORDER BY oxygen_delivery_device) AS oxygen_delivery_device_sequence FROM tv
""")
dv_oxygen_delivery_device.toPandas()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `date` cannot be resolved. Did you mean one of the following? [`id`, `value`, `charttime`, `patient_id`, `storetime`].; line 3 pos 42;
'WithCTE
:- 'CTERelationDef 3, false
:  +- 'SubqueryAlias tv
:     +- 'Project [patient_id#2496, encounter_id#2497, cast('date as timestamp_ntz) AS obs_time#2573, value#2500 AS oxygen_delivery_device#2574]
:        +- SubqueryAlias rv_oxygen_delivery_device
:           +- View (`rv_oxygen_delivery_device`, [id#2495,patient_id#2496,encounter_id#2497,charttime#2498,storetime#2499,value#2500])
:              +- Project [id#2495, patient_id#2496, encounter_id#2497, charttime#2498, storetime#2499, value#2500]
:                 +- Project [id#2495, patient_id#2496, encounter_id#2497, charttime#2498, storetime#2499, value#2500]
:                    +- Generate inline(array(struct(id, Observation#2487.id, patient_id, Observation#2487.subject.reference, encounter_id, 

In [ ]:
dv_oxygen_delivery_device.select("oxygen_delivery_device").distinct().show()